In [ ]:
# !pip install lxml requests beautifulsoup4

In [2]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
from tqdm import tqdm
import os

In [ ]:
# url = 'https://nomads.ncep.noaa.gov/pub/data/nccf/com/gfs/prod/'
url = "https://agroserver.ru/spros/agro/"

In [3]:
def get_files_grib(url=None):
    if url == None:
        url = "https://agroserver.ru/spros/agro/"
    response = requests.get(url)
    if response.status_code == 200:
        print(f"Status code=={response.status_code}")
        soup = BeautifulSoup(response.text, 'lxml')
        return soup
    else:
        return None
soup = get_files_grib()

Status code==200


In [ ]:

links_grib = []
for link in soup.find_all('a', href=True):
    # print(link['href'])
    if str(link).count('gfs')>0:
        links_grib.append(url + str(link['href']))
links_grib = links_grib[-2:-1]
links_grib = [links_grib[0] + i + '/atmos/' for i in ['00', '06', '12', '18']]
print(len(links_grib), links_grib[0])
grib_files_full_day = []
for item in tqdm(links_grib, total=len(links_grib)):
    print(item)
    soup = get_files_grib(item)
    # soup
    gribs_link = [item + i['href'] for i in soup.find_all('a', href=True)][2:]
    # gribs_link[0]
    grib_files = []
    for link in tqdm(gribs_link, total=len(gribs_link)):
        filename = link.split('/')[-1].split('.')
        if ('gfs' in filename) and (filename[2]=='pgrb2')\
            and (GRID_STEP in filename) and (len(filename)==5)\
                and (filename[-1].startswith('f')):
            # print(link)
            grib_files.append(link)
    # len(grib_files), grib_files[0]
    grib_files_full_day.append(grib_files[::SKIP])

In [ ]:
# grib_files_full_day

In [ ]:
import numpy as np
grib_files_full_day = sorted(list(np.asarray(grib_files_full_day).ravel()))
len(grib_files_full_day), grib_files_full_day[-1]

In [ ]:
def download_file(url, path_out=''):
    if path_out == '':
        os.makedirs(f"./tmp/{url.split('/')[-4].split('.')[-1]}/", exist_ok=True)
        local_filename = f"./tmp/{url.split('/')[-4].split('.')[-1]}/" + url.split('/')[-1]
    else:
        path_out = path_out + f"{url.split('/')[-4].split('.')[-1]}/"
        os.makedirs(path_out, exist_ok=True)
        local_filename = path_out + url.split('/')[-1]
    ### NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        LEN_CONT = int(r.headers['Content-length'])
        # print(LEN_CONT)
        # CHUNK = int(int(r.headers['Content-length']) / 100)
        CHUNK = 8192 * 2# 8192
        with open(local_filename, 'wb') as f:
            for chunk in tqdm(r.iter_content(chunk_size = CHUNK), total=int(LEN_CONT/CHUNK)): 
                f.write(chunk)
        f.close()
    # return local_filename

In [ ]:
for name_grib in tqdm(grib_files_full_day[::10], total=len(grib_files_full_day[::10])):
    print(name_grib)
    local_filename = download_file(name_grib, path_out='/home/sergio/Habr_fl/DZHALIL_METEO/GRIBS/')

In [ ]:
# ds = pygrib.open("./tmp/gfs.t00z.pgrb2.1p00.f000")